# Dash App Layout With Multiple Inputs

https://dash.plotly.com/basic-callbacks

In [ ]:
# importrequired libraries 
from jupyter_dash import JupyterDash
import dash_core_components as dcc # for graphs
import dash_html_components as html # for html
from dash.dependencies import Input, Output # for callbacks
import plotly.express as px
import pandas as pd

In [ ]:
# we'll use an external CSS style sheet
EXTERNAL_STYLESHEETS = ["https://codepen.io/chriddyp/pen/bWLwgP.css"]

In [ ]:
# read data of statistics for countries from 1952 to 2007, at every 5 year interval
# for every country and year: population, life expectency, GDP per capita
df = pd.read_csv("https://plotly.github.io/datasets/country_indicators.csv")
df

In [ ]:
available_indicators = df['Indicator Name'].unique()
available_indicators

In [ ]:
# initialize the dash app
app = JupyterDash(__name__, external_stylesheets = EXTERNAL_STYLESHEETS ) 

In [ ]:
# layout of the dashboard, without writing explicit html
app.layout = html.Div([
    html.Div([

        html.Div([
            dcc.Dropdown(
                id='xaxis-column',
                options=[{'label': i, 'value': i} for i in available_indicators],
                value='Fertility rate, total (births per woman)'
            ),
            dcc.RadioItems(
                id='xaxis-type',
                options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
                value='Linear',
                labelStyle={'display': 'inline-block'}
            )
        ],
        style={'width': '48%', 'display': 'inline-block'}),

        html.Div([
            dcc.Dropdown(
                id='yaxis-column',
                options=[{'label': i, 'value': i} for i in available_indicators],
                value='Life expectancy at birth, total (years)'
            ),
            dcc.RadioItems(
                id='yaxis-type',
                options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
                value='Linear',
                labelStyle={'display': 'inline-block'}
            )
        ],style={'width': '48%', 'float': 'right', 'display': 'inline-block'})
    ]),

    dcc.Graph(id='indicator-graphic'),

    dcc.Slider(
        id='year--slider',
        min=df['Year'].min(),
        max=df['Year'].max(),
        value=df['Year'].max(),
        marks={str(year): str(year) for year in df['Year'].unique()},
        step=None
    )
])

In [ ]:
# function ow to update the data at each callback
# note the code for function and the callback decorators must be together in the same cell
@app.callback(
    Output('indicator-graphic', 'figure'),
    Input('xaxis-column', 'value'),
    Input('yaxis-column', 'value'),
    Input('xaxis-type', 'value'),
    Input('yaxis-type', 'value'),
    Input('year--slider', 'value'))

def update_graph(xaxis_column_name, yaxis_column_name,
                 xaxis_type, yaxis_type,
                 year_value):
    dff = df[df['Year'] == year_value]

    fig = px.scatter(x=dff[dff['Indicator Name'] == xaxis_column_name]['Value'],
                     y=dff[dff['Indicator Name'] == yaxis_column_name]['Value'],
                     hover_name=dff[dff['Indicator Name'] == yaxis_column_name]['Country Name'])

    fig.update_layout(margin={'l': 40, 'b': 40, 't': 10, 'r': 0}, hovermode='closest')

    fig.update_xaxes(title=xaxis_column_name,
                     type='linear' if xaxis_type == 'Linear' else 'log')

    fig.update_yaxes(title=yaxis_column_name,
                     type='linear' if yaxis_type == 'Linear' else 'log')

    return fig

In [ ]:
# running the server 
app.run_server(mode = "inline")
#app.run_server(mode = "external")
#app.run_server(mode = "jupyterlab")